In [1]:
!pip install laser_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 895.1/895.1 kB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 36.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.3/288.3 kB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 20.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fairseq: filename=fairseq-0.12.2-cp310-cp310-linux_x86_64.whl size=11291809 sha256=25ceaaba018cc319f79626efa989583e9fbc355fc1ff47a8aa6cef3efa46d126
  Stored in directory: /root/.cache/pip/wheels/e4/35/55/9c66f65ec7c83fd6fbc2b9

In [2]:
!wget "https://raw.githubusercontent.com/NIXBLACK11/telegramOrganizationChatBot/main/v2/intents.json"

--2024-02-08 12:34:58--  https://raw.githubusercontent.com/NIXBLACK11/telegramOrganizationChatBot/main/v2/intents.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15941 (16K) [text/plain]
Saving to: ‘intents.json’

intents.json        100%[===================>]  15.57K  --.-KB/s    in 0s      

2024-02-08 12:34:58 (89.8 MB/s) - ‘intents.json’ saved [15941/15941]



In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import random
import json

In [4]:
from laser_encoders import LaserEncoderPipeline

In [5]:
encoder = LaserEncoderPipeline(lang="eng_Latn")

100%|██████████| 1.01M/1.01M [00:00<00:00, 31.4MB/s]
100%|██████████| 179M/179M [00:03<00:00, 58.7MB/s]
100%|██████████| 470k/470k [00:00<00:00, 24.6MB/s]


In [6]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    print("CUDA is available! Using GPU.")
else:
    device = torch.device('cpu')
    print("CUDA is not available. Using CPU.")

CUDA is available! Using GPU.


In [7]:
words = []
labels = []
docs_x = []
docs_y = []

In [8]:
# Load the intents JSON file
with open('intents.json') as file:
    data = json.load(file)

In [9]:
test_sentence = encoder.encode_sentences(["How are you"])[0]
print(test_sentence)
print(len(test_sentence))

[ 0.00115137 -0.00047817 -0.00038098 ... -0.00121862  0.0027636
  0.00175885]
1024


In [10]:
for intent in data['intents']:
    for pattern in intent['patterns']:
        processed_sentence = ''.join(char.lower() for char in pattern if char.isalnum() or char.isspace())
        tokenized_words = encoder.encode_sentences([pattern])[0]
        docs_x.append(tokenized_words)
        docs_y.append(intent['tag'])
    if intent['tag'] not in labels:
        labels.append(intent['tag'])

In [11]:
labels = sorted(labels)

In [12]:
training = []
output = []

In [13]:
for x, doc in enumerate(docs_x):
    output_row = [0] * len(labels)
    output_row[labels.index(docs_y[x])] = 1
    output.append(output_row)

In [14]:
training = np.array(docs_x)
output = np.array(output)

In [15]:
import torch
from torch.utils.data import Dataset

class ChatbotDataset(Dataset):
    def __init__(self, training, output):
        self.training = torch.tensor(training, dtype=torch.float32)
        self.output = torch.tensor(output, dtype=torch.float32)

    def __len__(self):
        return len(self.training)

    def __getitem__(self, idx):
        return self.training[idx], self.output[idx]

# Assuming you have 'training' and 'output' from your data processing code
# Instantiate the dataset
dataset = ChatbotDataset(training=training, output=output)

# Instantiate the DataLoader
batch_size = 32
train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=2)


In [16]:
# Define the neural network model using PyTorch
class ChatbotModel(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(ChatbotModel, self).__init__()
    self.l1 = nn.Linear(input_size, hidden_size)
    self.l2 = nn.Linear(hidden_size, hidden_size)
    self.l3 = nn.Linear(hidden_size, output_size)
    self.relu = nn.ReLU()

  def forward(self, x):
    out = self.l1(x)
    out = self.relu(out)
    out = self.l2(out)
    out = self.relu(out)
    out = self.l3(out)
    return out

In [17]:
input_size = len(training[0])
hidden_size = 512
output_size = len(output[0])
learning_rate = 0.001
num_epochs = 10

In [18]:
# Instantiate the model
model = ChatbotModel(input_size, hidden_size, output_size).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [19]:
def calculate_accuracy(model, dataloader, device):
    model.eval()  # Set the model to evaluation mode
    correct_predictions = 0
    total_samples = 0

    with torch.no_grad():
        for batch_inputs, batch_outputs in dataloader:
            batch_inputs = batch_inputs.to(device)
            batch_outputs = batch_outputs.to(device)

            # Forward pass
            outputs = model(batch_inputs)

            # Get predictions
            _, predicted_labels = torch.max(outputs, 1)
            batch_outputs = torch.argmax(batch_outputs, dim=-1)

            # Count correct predictions
            correct_predictions += (predicted_labels == batch_outputs).sum().item()
            total_samples += batch_outputs.size(0)

    accuracy = correct_predictions / total_samples
    model.train()  # Set the model back to training mode

    return accuracy

In [20]:
for epoch in range(num_epochs):
    for batch_inputs, batch_outputs in train_loader:

        batch_inputs = batch_inputs.to(device)
        batch_outputs = batch_outputs.to(device)
        # Zero the gradientsNeuralNetwork
        optimizer.zero_grad()

        # Forward pass
        outputs = model(batch_inputs)
        loss = criterion(outputs, batch_outputs)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

    if epoch % 1 == 0:
      train_accuracy = calculate_accuracy(model, train_loader, device)
      print(f"Epoch {epoch}/{num_epochs}, Loss: {loss.item()}, Training Accuracy: {train_accuracy * 100:.2f}%")

Epoch 0/10, Loss: 3.2865238189697266, Training Accuracy: 4.79%
Epoch 1/10, Loss: 3.4090986251831055, Training Accuracy: 17.47%
Epoch 2/10, Loss: 3.2634243965148926, Training Accuracy: 22.60%
Epoch 3/10, Loss: 2.8771555423736572, Training Accuracy: 27.40%
Epoch 4/10, Loss: 2.318315267562866, Training Accuracy: 41.78%
Epoch 5/10, Loss: 2.406074047088623, Training Accuracy: 49.32%
Epoch 6/10, Loss: 1.9173351526260376, Training Accuracy: 63.36%
Epoch 7/10, Loss: 2.004138231277466, Training Accuracy: 60.62%
Epoch 8/10, Loss: 1.7296844720840454, Training Accuracy: 58.90%
Epoch 9/10, Loss: 1.1002148389816284, Training Accuracy: 63.36%


In [21]:
# Save the model
torch.save(model.state_dict(), 'model.pth')

In [22]:
state_dict = torch.load('model.pth')
model = ChatbotModel(input_size, hidden_size, output_size).to(device)
model.load_state_dict(state_dict)
model.eval()


ChatbotModel(
  (l1): Linear(in_features=1024, out_features=512, bias=True)
  (l2): Linear(in_features=512, out_features=512, bias=True)
  (l3): Linear(in_features=512, out_features=27, bias=True)
  (relu): ReLU()
)

In [23]:
model.eval()

ChatbotModel(
  (l1): Linear(in_features=1024, out_features=512, bias=True)
  (l2): Linear(in_features=512, out_features=512, bias=True)
  (l3): Linear(in_features=512, out_features=27, bias=True)
  (relu): ReLU()
)

In [24]:
def process_input(user_input):
    # tokenized_input = nltk.word_tokenize(user_input)
    # tokenized_input = [stemmer.stem(word.lower()) for word in tokenized_input]
    user_input = user_input.lower()
    input_bag = encoder.encode_sentences([user_input])[0]

    input_bag = torch.tensor(input_bag, dtype=torch.float32).to(device)
    input_bag = input_bag.unsqueeze(0)  # Add a batch dimension

    results = model(input_bag)
    results_index = torch.argmax(results).item()
    tag = labels[results_index]

    for intent in data['intents']:
        if intent['tag'] == tag:
            return random.choice(intent['responses'])

In [25]:
process_input("")

'Talk to you later'

In [26]:
sentences = ["Hi", "who made you", "what are the bank timings", "what is my account balance", "where is the bank located",
              "नमस्ते", "आपको किसने बन", "बैंक का समय क्या", "मेरे खाते का शेष राशि क्", "बैंक कहाँ स्थित है?",
              "Hola", "¿Quién te creó?", "¿Cuáles son los horarios del banco?", "¿Cuál es el saldo de mi cuenta?", "¿Dónde está ubicado el banco?",
              "what documents are required to open an account", "खाता खोलने के लिए कौन-कौन से दस्तावेज़ आ", "¿Qué documentos se requieren para abrir una cuenta?"]

for sentence in sentences:
  print(f"You: {sentence}")
  print(f"Bot: {process_input(sentence)}")

You: Hi
Bot: Sad to see you go :(
You: who made you
Bot: My creator is NIXBLACK
You: what are the bank timings
Bot: Our bank follows a holiday schedule provided in advance. For details, refer to the official bank calendar or contact our customer service for information about upcoming holidays.
You: what is my account balance
Bot: balance_enquiry
You: where is the bank located
Bot: The bank is located at Nainital, Uttarakhand
You: नमस्ते
Bot: Come back soon
You: आपको किसने बन
Bot: I am your Chatbanker.
You: बैंक का समय क्या
Bot: The bank is open from 9am to 5pm Monday-Friday and from 9am to 12pm on Saturdays!
You: मेरे खाते का शेष राशि क्
Bot: Our bank follows a holiday schedule provided in advance. For details, refer to the official bank calendar or contact our customer service for information about upcoming holidays.
You: बैंक कहाँ स्थित है?
Bot: The bank is located at Nainital, Uttarakhand
You: Hola
Bot: Sad to see you go :(
You: ¿Quién te creó?
Bot: My creator is NIXBLACK
You: ¿Cuál

In [27]:
print("Bank Chatbot - Ask me anything!")
while True:
    user_input = input("You: ")
    if user_input.lower() == 'quit':
        break

    response = process_input(user_input)
    print("Bot: " + response)

Bank Chatbot - Ask me anything!
You: hello
Bot: Sad to see you go :(
You: what
Bot: Sad to see you go :(
You: eejdsknscd
Bot: You can contact the bank at: BANK_PHONE_NUMBER
You: tell me the account balance
Bot: balance_enquiry
You: hello
Bot: Come back soon


KeyboardInterrupt: Interrupted by user

In [ ]:
import pickle

input_size = len(training[0])
hidden_size = 512
output_size = len(output[0])

sizes = [input_size, output_size, hidden_size]

with open('sizes.pkl', 'wb') as file:
    loaded_array = pickle.dump(sizes, file)


with open('lables.pkl', 'wb') as file:
    loaded_array = pickle.dump(labels, file)

In [ ]:
!ls -lah